In [1]:
pip install -U finance_datareader


In [2]:
import FinanceDataReader  as fdr
import matplotlib.pyplot as pltfrom 
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils

import numpy
import tensorflow as tf
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [0]:
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt


In [4]:
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=1000,test_split=0.2)


2113536/2110848 [==============================] - 1s 0us/step


In [0]:
Y_train = np_utils.to_categorical(Y_train)

In [6]:
X_train

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]),
       list([1, 2, 699, 2, 2, 56, 2, 2, 9, 56, 2, 2, 81, 5, 2, 57, 366, 737, 132, 20, 2, 7, 2, 49, 2, 2, 2, 2, 699, 2, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2, 2, 2, 775, 7, 48, 34, 191, 44, 35, 2, 505, 17, 12]),
       list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 2, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 2, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110, 59, 89, 90, 67, 2, 55, 2, 92, 617, 80, 2, 46, 905, 220, 13,

In [0]:
import requests

# STEP 01: Generate OTP
gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
gen_otp_data = {
    'name':'fileDown',
    'filetype':'xls',
    'url':'MKD/04/0404/04040200/mkd04040200_01',
    'market_gubun':'ALL', #시장구분: ALL=전체, STK=코스피, KSQ=코스피, KNX=코넥스
    'indx_ind_cd':'', 
    'sect_tp_cd':'',
    'schdate':'20180601',
    'pagePath':'/contents/MKD/04/0404/04040200/MKD04040200.jsp',
}
    
r = requests.post(gen_otp_url, gen_otp_data)
code = r.content

# STEP 02: download
down_url = 'http://file.krx.co.kr/download.jspx'
down_data = {
    'code': code,
}

headers = {'Referer': 'http://marketdata.krx.co.kr'}
r = requests.post(down_url, down_data, headers=headers)
with open("marcap_data.xls", 'wb') as f:
    f.write(r.content)

In [0]:
import pandas as pd

pd.read_excel('marcap_data.xls').head(3)

,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수,외국인 보유주식수,외국인 지분율(%)
0,005930,삼성전자,"51,300",600,1.2,"13,038,499","665,063,170,392","50,500","51,700","49,950","329,311,357,110,000",17.10,"6,419,324,700","3,405,723,291",53.05
1,000660,SK하이닉스,"91,400","-2,000",-2.1,"4,790,644","435,439,684,800","90,800","92,200","89,600","66,539,416,161,000",3.46,"728,002,365","368,274,204",50.59
2,005935,삼성전자우,"40,700",200,0.5,"976,217","39,713,003,950","40,600","40,900","40,250","36,777,700,300,000",1.91,"903,629,000","760,518,760",84.16


In [0]:
sams = fdr.DataReader('005930', '2016-01-01')
naver = fdr.DataReader('035420', '2016-01-01')
kakao = fdr.DataReader('035720', '2016-01-01')
lg = fdr.DataReader('066570', '2016-01-01')

sams = sams.drop(['Volume', 'Change'], axis = 1)
naver = naver.drop(['Volume', 'Change'], axis = 1)
kakao = kakao.drop(['Volume', 'Change'], axis = 1)
lg = lg.drop(['Volume', 'Change'], axis = 1)

sams.head(5)

,Open,High,Low,Close
Date,,,,
2016-01-04,25200,25200,24100,24100
2016-01-05,24040,24360,23720,24160
2016-01-06,24160,24160,23360,23500
2016-01-07,23320,23660,23020,23260
2016-01-08,23260,23720,23260,23420


In [0]:
samsung_data = []
naver_data = []
kakao_data = []
lg_data = []

for s, n , k , l in zip(sams.values, naver.values, kakao.values, lg.values):
  samsung_data.append(s)
  naver_data.append(n)
  kakao_data.append(k)
  lg_data.append(l)

In [0]:
len(naver_data)

906

In [0]:
import numpy as np

In [0]:
arr = np.empty((0,3), int)
arr

array([], shape=(0, 3), dtype=int64)

In [0]:
x_data = np.empty((0,3))
y_data = np.empty((0,3))
#0 삼성 1 네이버 2 카카오 3 엘지
for n in range(906):
  x_data = np.append(x_data, (samsung_data[n]/1000).tolist())
  x_data = np.append(x_data, (naver_data[n]/1000).tolist())
  x_data = np.append(x_data, (kakao_data[n]/1000).tolist())
  x_data = np.append(x_data, (lg_data[n]/1000).tolist())
  # x_data.append((samsung_data[n]/1000).tolist())
  # x_data.append((naver_data[n]/1000).tolist())
  # x_data.append((kakao_data[n]/1000).tolist())
  # x_data.append((lg_data[n]/1000).tolist())
  y_data.append(0)
  y_data.append(1)
  y_data.append(2)
  y_data.append(3)
  
  

In [0]:
x_data

array([25.2 , 25.2 , 24.1 , ..., 47.4 , 46.8 , 47.15])

In [0]:
x_train = x_data[:3200]
y_train = y_data[:3200]
x_test = x_data[3201:]
y_test = y_data[3201:]

In [0]:
x_train

[array([25.2, 25.2, 24.1, 24.1]),
 array([130.184, 131.786, 126.579, 126.58 ]),
 array([117.5, 117.5, 115.3, 115.3]),
 array([53.9, 53.9, 52.5, 52.5]),
 array([24.04, 24.36, 23.72, 24.16]),
 array([126.779, 128.582, 126.379, 127.581]),
 array([114.1, 117.5, 113.8, 117.1]),
 array([52.1, 54.5, 51.9, 54.3]),
 array([24.16, 24.16, 23.36, 23.5 ]),
 array([129.183, 129.183, 124.576, 124.777]),
 array([118. , 121.1, 117.3, 119.1]),
 array([54.1, 55.2, 53.7, 54.7]),
 array([23.32, 23.66, 23.02, 23.26]),
 array([125.377, 132.187, 125.377, 130.585]),
 array([118.4, 120.5, 116.6, 117.3]),
 array([55.2, 56.9, 54.9, 55. ]),
 array([23.26, 23.72, 23.26, 23.42]),
 array([130.585, 131.987, 128.582, 130.585]),
 array([115.3, 115.9, 113.7, 115.2]),
 array([54.5, 56.1, 54.1, 55.9]),
 array([23.12, 23.32, 22.92, 23.04]),
 array([129.183, 130.384, 125.377, 125.778]),
 array([114.8, 120.4, 114.1, 114.7]),
 array([55.7, 56.6, 55.4, 55.5]),
 array([22.96, 23.32, 22.88, 22.92]),
 array([127.38 , 130.985, 126.